In [1]:
import pandas as pd
import numpy as np
from collections import Counter
from datetime import datetime
import csv
from statistics import mode

In [2]:
data_exl_500 = pd.read_csv("../../gen/input/amazon.csv", sep = ";", header = None, usecols=range(16))
len(data_exl_500)

C:\Users\janva\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


165475

In [3]:
data_500 = pd.read_csv("../../data/amazon_500+.csv", sep = ";", usecols=range(16))
len(data_500)

41933

In [4]:
column_names = ["asin_url", "brand", "model", "link", "product_title",
             "name", "rating", "review_title", "country", "date_amazon",
             "specs", "first_spec", "review", "helpful", "picture",
            "verified"]
data_exl_500.columns = column_names
data_500.columns = column_names

In [6]:
data_raw = pd.concat([data_exl_500, data_500])

In [18]:
pd.concat([data_raw, data_500])[pd.concat([data_raw, data_500]).duplicated(subset=['name', 'rating', 'date', 'review', 'specs'])]

,asin_url,brand,model,link,product_title,name,rating,review_title,country,date,specs,first_spec,review,helpful,picture,verified
13282,https://www.amazon.com/dp/B097CNBDX2,SAMSUNG,Z Fold 3 5G,https://amazon.com/Samsung-Electronics-Unlocke...,"SAMSUNG Galaxy Z Fold 3 5G Cell Phone, Factory...",Andrei,1.0,Screen WILL Brake,the United States,"January 25, 2022",Style: Z Fold 3Color: Phantom BlackSize: 256GB,Z,2000$ pile of trash screen will brake samsung ...,7,no,no
13443,https://www.amazon.com/dp/B07P976BBH,Apple,IPhone XR,https://amazon.com/Apple-iPhone-Fully-Unlocked...,"Apple iPhone XR, US Version, 128GB, Black - Un...",Terri,4.0,1st Time Apple iPhone User,the United States,"August 10, 2020",Size: 256GBColor: RedService Provider: Unlocke...,256GBColor:,The phone I received was by all estimations br...,14,no,yes
13444,https://www.amazon.com/dp/B07P976BBH,Apple,IPhone XR,https://amazon.com/Apple-iPhone-Fully-Unlocked...,"Apple iPhone XR, US Version, 128GB, Black - Un...",S Henry,5.0,Perfect First iPhone!,the United States,"August 1, 2022",Size: 256GBColor: CoralService Provider: Unloc...,256GBColor:,I’ve been an Android user since the beginning ...,28,1,yes
13445,https://www.amazon.com/dp/B07P976BBH,Apple,IPhone XR,https://amazon.com/Apple-iPhone-Fully-Unlocked...,"Apple iPhone XR, US Version, 128GB, Black - Un...",Tim,5.0,As advertised,the United States,"September 10, 2022",Size: 128GBColor: RedService Provider: Unlocke...,128GBColor:,This was a great gift for my daughter who uses...,none,no,yes
13446,https://www.amazon.com/dp/B07P976BBH,Apple,IPhone XR,https://amazon.com/Apple-iPhone-Fully-Unlocked...,"Apple iPhone XR, US Version, 128GB, Black - Un...",JD,4.0,So far so good.,the United States,"August 27, 2022",Size: 128GBColor: BlackService Provider: Unloc...,128GBColor:,Bought this as my daughters first phone so she...,10,1,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37870,https://www.amazon.com/Samsung-Factory-Unlocke...,samsung,Samsung Galaxy S10 Plus,https://www.amazon.com/Samsung-Factory-Unlocke...,"Samsung Galaxy S10+, 128GB, Prism Black - Unlo...",Jimmy,5,Excellent phone,the United States 🇺🇸,"October 22, 2019","['Size: 128GB', 'Color: Prism Black', 'Service...",Prism Black,This phone works great.,NaN,no,yes
37871,https://www.amazon.com/Samsung-Factory-Unlocke...,samsung,Samsung Galaxy S10 Plus,https://www.amazon.com/Samsung-Factory-Unlocke...,"Samsung Galaxy S10+, 128GB, Prism Black - Unlo...",Kanishka Ponnamperuma,5,Amazing Refurbished S10+,the United States 🇺🇸,"August 12, 2021","['Size: 128GB', 'Color: Prism Black', 'Service...",Prism Black,I use the S10+ as my secondary phone to my iPh...,NaN,no,yes
38227,https://www.amazon.com/Samsung-Factory-Unlocke...,samsung,Samsung Galaxy S10 Plus,https://www.amazon.com/Samsung-Factory-Unlocke...,"Samsung Galaxy S10+, 128GB, Prism Black - Unlo...",Adolfo Linares Villa,5,Excelente,Mexico,"April 17, 2020","['Size: 128GB', 'Color: Prism Black', 'Service...",Prism Black,Muy buen producto,NaN,no,yes
38707,https://www.amazon.com/Samsung-Galaxy-SM-G9650...,samsung,Galaxy S9 Plus,https://www.amazon.com/Samsung-Galaxy-SM-G9650...,"Samsung Galaxy S9+, 64GB, Midnight Black - Ful...",Shawn Fenner,4,good phone eventually,the United States 🇺🇸,"April 27, 2020","['Size: 64GB', 'Color: Lilac Purple', 'Service...",Lilac Purple,I bought this phone for use with straight talk...,NaN,no,yes


# Removing data

In [8]:
data_usa = data_raw[data_raw.country=="the United States"]
print(len(data_raw))
print(len(data_usa))
print(len(data_raw) - len(data_usa))

207408
150580
56828


In [29]:
data_usa.iloc[-1,]['asin_url']

'https://www.amazon.com/Samsung-Galaxy-SM-G9650-Factory-Unlocked/product-reviews/B07D3Z9ZST/ref=cm_cr_arp_d_paging_btm_next_2?ie=UTF8&reviewerType=all_reviews&pageNumber=304'

In [9]:
asins = []
for asin_url in data_usa['asin_url']:
    split_asin_url = asin_url.split('/')
    if split_asin_url[3] == "dp":
        asin = split_asin_url[-1]
    else:
        asin = split_asin_url[-2]
    asins.append(asin)
data_usa['asin_from_link'] = asins

<ipython-input-9-2df3ad4f8abe>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_usa['asin_from_link'] = asins


In [10]:
print(len(data_usa))
data_no_dupli = data_usa[~data_usa.duplicated(subset=['name', 'rating', 'date', 'review', 'specs'])]
print(len(data_usa) - len(data_no_dupli))
print(len(data_no_dupli))

150580
6350
144230


In [12]:
#Date from amazon notation to datetime
date_trans = []
for row in data_no_dupli['date_amazon'].tolist():
    date_trans.append(datetime.strptime(row, "%B %d, %Y"))
    
data_no_dupli['date'] = date

<ipython-input-12-66fee15b4b5a>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_no_dupli['date_trans'] = date_trans


In [14]:
print(len(data_no_dupli))
data_after_date = data_no_dupli[data_no_dupli.date <= "2022-10-08"]
print(len(data_no_dupli) - len(data_after_date))
print(len(data_after_date))

144230
722
143508


In [15]:
asin_to_be_removed = ['https://www.amazon.com/dp/B085P16MD8',
                     'https://www.amazon.com/dp/B09M8DRDL7',
                     'https://www.amazon.com/dp/B09LYMZ49Q',
                     'https://www.amazon.com/dp/B07P5P9X78',
                     'https://www.amazon.com/dp/B08H8R75ZT',
                     'https://www.amazon.com/dp/B08C6YB4LY',
                     'https://www.amazon.com/dp/B079H86JGH',
                     'https://www.amazon.com/dp/B09742XJB1',
                     'https://www.amazon.com/dp/B0998CV6P6',
                     'https://www.amazon.com/dp/B077RM9WJB',
                     'https://www.amazon.com/dp/B096BG51BD',
                     'https://www.amazon.com/dp/B07W81LNTD',
                     'https://www.amazon.com/dp/B0B7TLY72K',
                     'https://www.amazon.com/dp/B097HFC8TB',
                     'https://www.amazon.com/dp/B09BQBH86J',
                     'https://www.amazon.com/dp/B06WGL82S6',
                     'https://www.amazon.com/dp/B07TS37SR1',
                     'https://www.amazon.com/dp/B0841S1XYH',
                     'https://www.amazon.com/dp/B00HS3F8TK',
                     'https://www.amazon.com/dp/B08GM7DRXM',
                     'https://www.amazon.com/dp/B084BQK3HP',
                     'https://www.amazon.com/dp/B07PLSTM1K',
                     'https://www.amazon.com/dp/B07BXYVHT5',
                     'https://www.amazon.com/dp/B08TGX6LCR',
                     'https://www.amazon.com/dp/B07B1VN8ZL',
                     'https://www.amazon.com/dp/B086NPRNHH',
                     'https://www.amazon.com/dp/B08G4K47GR',
                     'https://www.amazon.com/dp/B006BEJIUC',
                     'https://www.amazon.com/dp/B092JG5NL1',
                     'https://www.amazon.com/dp/B09CYCVSZ8',
                     'https://www.amazon.com/dp/B01JIP8WPM',
                     'https://www.amazon.com/dp/B0858QDP5C',
                     'https://www.amazon.com/dp/B09TPZJB1P',
                     'https://www.amazon.com/dp/B074HY3LRB',
                     'https://www.amazon.com/dp/B085YC83JF',
                     'https://www.amazon.com/dp/B084GXS58R',
                     'https://www.amazon.com/dp/B08WVF25JC',
                     'https://www.amazon.com/dp/B0759QPPKN',
                      'https://www.amazon.com/dp/B09JCH6V5C'
                     ]

In [18]:
print(len(data_after_date))
data = data_after_date[~data_after_date['asin_url'].isin(asin_to_be_removed)] #3226 
print(len(data_after_date) - len(data))
print(len(data))

143508
2913
140595


# Variable transformation

In [19]:
#to integer
integer_ratings = []
for index, row in data.iterrows():
    if row['rating'] == 'none':
        integer_ratings.append(None)
    else:
        integer_ratings.append(float(row['rating']))
data['rating'] = integer_ratings

<ipython-input-19-a9735a962561>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['rating'] = integer_ratings


# Colors
The following code looks which color belongs to which smartphone and groups them under secondary colors.

In [20]:
#colours
colors = ["blue", "green", "orange", "yellow", "purple", "brown", "gray","grey", "red", "black",
          "gold", "pink", "silver","white", "coral", "sage", "emerald", "glow", "bronze",
          "charcoal","violet", "graphite","burgundy", "cloud", "navy", "mint", "lavender",
          "phantom titanium", "olive", "peach", "night", "moon", "lime", "apricot", "starlight",
         "indigo", "sand", "rock", "sc490", "morning mist", "cyan", "winter mist", "mercury",
         "frosted", "cream", "electric shadow", "dark", "raspberry", "metallic", "blush"]
variant_colors = []

i = 1
for index, row in data.iterrows():
    variant_color = []
    for color in colors:
        #Search for colors in the specs column
        if str(row['specs']).lower().find(color) != -1:
            variant_color.append(color)
        #If the word color is not found in specs, look in the title of the smartphone
        elif str(row['specs']).lower().find('color') == -1:
            if str(row['product_title']).lower().find(color) != -1:
                variant_color.append(color)           
    
    #If no color could be found, append none insteada
    if len(variant_color) == 0:
        variant_color.append('none')
    else:
        #do some corrections
        if variant_color[0] == "coral" or variant_color[0] == "peach" or variant_color[0] == "apricot":
            variant_color[0] = "orange"
        elif variant_color[0] == "sage" or variant_color[0] == "emerald" or variant_color[0] == "mint" or variant_color[0] == "olive"  or variant_color[0] == "lime":
            variant_color[0] = "green"
        elif variant_color[0] == "glow":
            variant_color[0] = "mixed"
        elif variant_color[0] == "bronze":
            variant_color[0] = "brown"
        elif variant_color[0] == "charcoal" or variant_color[0] == "graphite" or variant_color[0] == "phantom titanium" or variant_color[0] == "night" or variant_color[0] == "sc490" or variant_color[0] == "dark":
            variant_color[0] = "black"
        elif variant_color[0] == "violet" or variant_color[0] == "lavender" or variant_color[0] == "indigo":
            variant_color[0] = "purple"
        elif variant_color[0] == "burgundy":
            variant_color[0] = "red"
        elif variant_color[0] == "cloud" or variant_color[0] == "navy" or variant_color[0] == "winter mist" or variant_color[0] == "frosted":
            variant_color[0] = "blue"
        elif variant_color[0] == "grey" or variant_color[0] == "moon" or variant_color[0] == "rock" or variant_color[0] == "morning mist" or variant_color[0] == "electric shadow" or variant_color[0] == "metallic":
            variant_color[0] = "gray"
        elif variant_color[0] == "starlight" or variant_color[0] == "mercury":
            variant_color[0] = "silver" 
        elif variant_color[0] == "sand":
            variant_color[0] = "yellow" 
        elif variant_color[0] == "blush" or variant_color[0] == "raspberry":
            variant_color[0] = "pink" 
    #others category        
    variant_colors.append(variant_color[0])
    i += 1
    if i%1000 == 0:
        print(f"{round((i/len(data))*100,2)}%")
data['variant'] = variant_colors

0.71%
1.42%
2.13%
2.85%
3.56%
4.27%
4.98%
5.69%
6.4%
7.11%
7.82%
8.54%
9.25%
9.96%
10.67%
11.38%
12.09%
12.8%
13.51%
14.23%
14.94%
15.65%
16.36%
17.07%
17.78%
18.49%
19.2%
19.92%
20.63%
21.34%
22.05%
22.76%
23.47%
24.18%
24.89%
25.61%
26.32%
27.03%
27.74%
28.45%
29.16%
29.87%
30.58%
31.3%
32.01%
32.72%
33.43%
34.14%
34.85%
35.56%
36.27%
36.99%
37.7%
38.41%
39.12%
39.83%
40.54%
41.25%
41.96%
42.68%
43.39%
44.1%
44.81%
45.52%
46.23%
46.94%
47.65%
48.37%
49.08%
49.79%
50.5%
51.21%
51.92%
52.63%
53.34%
54.06%
54.77%
55.48%
56.19%
56.9%
57.61%
58.32%
59.03%
59.75%
60.46%
61.17%
61.88%
62.59%
63.3%
64.01%
64.72%
65.44%
66.15%
66.86%
67.57%
68.28%
68.99%
69.7%
70.42%
71.13%
71.84%
72.55%
73.26%
73.97%
74.68%
75.39%
76.11%
76.82%
77.53%
78.24%
78.95%
79.66%
80.37%
81.08%
81.8%
82.51%
83.22%
83.93%
84.64%
85.35%
86.06%
86.77%
87.49%
88.2%
88.91%
89.62%
90.33%
91.04%
91.75%
92.46%
93.18%
93.89%
94.6%
95.31%
96.02%
96.73%
97.44%
98.15%
98.87%
99.58%


<ipython-input-20-49f0754c49cb>:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['variant'] = variant_colors


In [24]:
data

,asin_url,brand,model,link,product_title,name,rating,review_title,country,date,specs,first_spec,review,helpful,picture,verified,date_trans,variant
0,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",wp,4.0,Works great.,the United States,"August 27, 2022",Size: 64GBColor: BlackService Provider: T-Mobi...,64GBColor:,Was skeptical to get one a renewed iPhone . De...,NaN,NaN,NaN,2022-08-27,black
1,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",J. R.,4.0,So far so good!,the United States,"September 5, 2022",Size: 128GBColor: WhiteService Provider: Unloc...,128GBColor:,I wasn’t sure what to expect from a refurbishe...,NaN,NaN,NaN,2022-09-05,white
2,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Dov,4.0,Battery life is not the best but the phone is ...,the United States,"August 18, 2022",Size: 64GBColor: BlackService Provider: Unlock...,64GBColor:,Apple phones are always a great phone his phon...,NaN,NaN,NaN,2022-08-18,black
3,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Jerome W,5.0,GREAT PRODUCT - IPhone 11,the United States,"August 26, 2022",Size: 256GBColor: WhiteService Provider: Unloc...,256GBColor:,The phone arrived on time and I was very nervo...,NaN,NaN,NaN,2022-08-26,white
4,https://www.amazon.com/dp/B07ZPKN6YR,Apple,IPhone 11,https://amazon.com/Apple-iPhone-11-64GB-Black/...,"Apple iPhone 11, 64GB, Black - Unlocked (Renew...",Katie Sisemore,5.0,The phone worked good,the United States,"September 7, 2022",Size: 64GBColor: PurpleService Provider: Unloc...,64GBColor:,The phone when I got it look just like new I w...,NaN,NaN,NaN,2022-09-07,purple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165470,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Shelly manning,1.0,this phone is only 3g. does NOT have 4g and mo...,the United States 🇺🇸,"October 6, 2021",['Color: Orange'],Orange,phone can NOT be used with every carrier.,none,no,yes,2021-10-06,orange
165471,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Abubakar Jalloh,5.0,Very strong body.,the United States 🇺🇸,"February 4, 2020",['Color: Orange'],Orange,I like the body of the phone.,1,no,yes,2020-02-04,orange
165472,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Janet Decker,1.0,Phone not unlocked,the United States 🇺🇸,"April 19, 2021",['Color: Orange'],Orange,Phone not unlocked as stated. 3g will be obsol...,none,no,yes,2021-04-19,orange
165473,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,Ulefone,NaN,https://www.amazon.com/Ulefone-Armor-X6-Waterp...,"Rugged Phones Unlocked, Ulefone Armor X6 (2022...",Kelly D.,1.0,DO NOT ORDER THIS PHONE!,the United States 🇺🇸,"June 13, 2020",['Color: Orange'],Orange,This phone only lasts 2 months before it dies!...,3,no,yes,2020-06-13,orange


In [22]:
color_asin_correction = {'https://www.amazon.com/dp/B098KF1G4Q' : 'black',
                         'https://www.amazon.com/dp/B08V1X89P9' : 'black',
                         'https://www.amazon.com/dp/B091JHNTNH' : 'gray',
                        'https://www.amazon.com/dp/B08CJDTDPL' : 'mixed',
                        'https://www.amazon.com/dp/B0B3B8VY4D' : 'cyan',
                        'https://www.amazon.com/dp/B08HC5GMK2' : 'black',
                        'https://www.amazon.com/dp/B08FMX83BL' : 'blue',
                        'https://www.amazon.com/dp/B09CYQKVNS' : 'black',
                        'https://www.amazon.com/dp/B0967RGC2H' : 'black',
                        'https://www.amazon.com/dp/B078P38R8L' : 'white',
                        'https://www.amazon.com/dp/B07PX3ZRJ6' : 'cyan',
                        'https://www.amazon.com/dp/B0933K762G' : 'brown',
                        'https://www.amazon.com/dp/B0B5DP53D9' : 'black',
                        'https://www.amazon.com/dp/B088VSPDFC' : 'black',
                        'https://www.amazon.com/dp/B08XB51L99' : 'cyan',
                        'https://www.amazon.com/dp/B09H8W169M' : 'black',
                        'https://www.amazon.com/dp/B0747WWP4D' : 'black',
                        'https://www.amazon.com/dp/B0985JTJD8' : 'blue',
                        'https://www.amazon.com/dp/B09DJ2P1SF' : 'black',
                        'https://www.amazon.com/dp/B09BM2FZ3M' : 'blue',
                        'https://www.amazon.com/dp/B08YR66D1H' : 'black',
                        'https://www.amazon.com/dp/B09GM4HR23' : 'gray',
                        'https://www.amazon.com/dp/B0B6FNQ3B6' : 'black',
                        'https://www.amazon.com/dp/B098LXDNZ4' : 'purple',
                        'https://www.amazon.com/dp/B09352C4XY' : 'black',
                        'https://www.amazon.com/dp/B07MQVW2T6' : 'blue',
                        'https://www.amazon.com/dp/B08LXKCCGL' : 'black',
                        'https://www.amazon.com/dp/B091YC2SJZ' : 'black',
                        'https://www.amazon.com/dp/B07CCNTHYH' : 'black',
                        'https://www.amazon.com/dp/B09YY38YD7' : 'purple',
                        'https://www.amazon.com/dp/B08GL48SYJ' : 'black',
                        'https://www.amazon.com/dp/B09KCNZKX3' : 'black',
                        'https://www.amazon.com/dp/B09S5MJLF6' : 'gray',
                        'https://www.amazon.com/dp/B083JN4KJX' : 'black',
                        'https://www.amazon.com/dp/B07ZZGT86P' : 'black',
                        'https://www.amazon.com/dp/B08JLT512D' : 'silver',
                        'https://www.amazon.com/dp/B093DXMSZB' : 'mixed',
                        'https://www.amazon.com/dp/B08W1RBJ63' : 'black',
                        'https://www.amazon.com/dp/B0B2RYBFBV' : 'blue',
                        'https://www.amazon.com/dp/B093XYRGFN' : 'black',
                        'https://www.amazon.com/dp/B09CL3HZFB' : 'mixed',
                        'https://www.amazon.com/dp/B0B712XJRK' : 'black',
                        'https://www.amazon.com/dp/B0B2VSQP3Y' : 'black',
                        'https://www.amazon.com/dp/B08FSP7C4L' : 'black',
                        'https://www.amazon.com/dp/B07DWWKGW8' : 'black',
                        'https://www.amazon.com/dp/B07KBB83N8' : 'gray',
                        'https://www.amazon.com/dp/B09CKZ9TRQ' : 'black',
                        'https://www.amazon.com/dp/B08RCV9CQQ' : 'mixed',
                        'https://www.amazon.com/dp/B07MBWWM25' : 'black',
                        'https://www.amazon.com/dp/B073TPZ48L' : 'black',
                        'https://www.amazon.com/dp/B09BVCDM6N' : 'mixed',
                        'https://www.amazon.com/dp/B09RP6SDDC' : 'black',
                        'https://www.amazon.com/dp/B09T95W93M' : 'black',
                        'https://www.amazon.com/dp/B08K2K831J' : 'black',
                        'https://www.amazon.com/dp/B06Y6J869C' : 'black',
                        'https://www.amazon.com/dp/B094VQBZNF' : 'mixed',
                        'https://www.amazon.com/dp/B01N1Q6V7K' : 'black',
                        'https://www.amazon.com/dp/B09963SWCY' : 'black',
                        'https://www.amazon.com/dp/B07PFQNNJT' : 'mixed'}

In [33]:
#Find missing colors
print(len(data[data['variant'] == 'none']))
print(data[data['variant'] == 'none']['specs'].unique())
print(data[data['variant'] == 'none']['asin_url'].unique())
print(data[data.variant == 'none'][['asin_url', 'specs']].drop_duplicates())

data.loc[(data.asin_url == "https://www.amazon.com/dp/B0855MYTY1") & (data.specs == "['Color: Titanium']"), 'variant'] = 'gray'

529
['none']
['https://www.amazon.com/dp/B08FRQMW31'
 'https://www.amazon.com/dp/B07K2K9WDL'
 'https://www.amazon.com/dp/B073KRVVRF'
 'https://www.amazon.com/dp/B077H73WSR'
 'https://www.amazon.com/dp/B08SYQ5WDS'
 'https://www.amazon.com/dp/B07GZLLTWC'
 'https://www.amazon.com/dp/B09DPVDWG7'
 'https://www.amazon.com/dp/B09H2L31NL']
                                   asin_url specs
10593  https://www.amazon.com/dp/B08FRQMW31  none
11470  https://www.amazon.com/dp/B07K2K9WDL  none
14669  https://www.amazon.com/dp/B073KRVVRF  none
25186  https://www.amazon.com/dp/B077H73WSR  none
31236  https://www.amazon.com/dp/B08SYQ5WDS  none
58875  https://www.amazon.com/dp/B07GZLLTWC  none
65898  https://www.amazon.com/dp/B09DPVDWG7  none
72000  https://www.amazon.com/dp/B09H2L31NL  none


C:\Users\janva\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [23]:
#Looping through rows to replace incorrect
variant_correction = []
for index, row in data.iterrows():
    if str(row['asin_url']) in list(color_asin_correction.keys()) and str(row['variant']) == 'none':
        variant = color_asin_correction[str(row['asin_url'])]
    else:
        variant = row['variant']
    variant_correction.append(variant)
data['variant'] = variant_correction

<ipython-input-23-1e78f063c332>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['variant'] = variant_correction


In [34]:
print(len(data[data['variant'] == 'none']))
print(variant_correction.count('none'))

529
563


# Renewed variable

In [35]:
#renewed variable ADD SPECS
renewed = []
for index, row in data.iterrows():
    if str(row['specs']).lower().find('renewed') != -1:
        renewed.append("yes")
    else:
        if str(row['product_title']).lower().find('renewed') != -1:
            renewed.append("yes")
        else:
            renewed.append("no")
data['renewed'] = renewed

<ipython-input-35-d831f468f2d8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['renewed'] = renewed


# Brands
Correction to the other brand column since that sometimes shows the provider etc.

In [36]:
brands = ['apple', 'iphone', 'google', 'samsung', 'motorola',
          'oneplus', 'tcl', 'blu ', 'jitterbug', 'zte', 'huawei',
          'nokia','sony', 'lg', 'htc', 'lenovo', 'xiaomi',
         'honor', 'oppo', 'realme', 'vivo', 'meizu', 'blackberry',
         'asus', 'alcatel', 'microsoft', 'energizer', 'cat', 'sharp',
         'ulefone', 'panasonic', 'nuu', 'oukitel', 'blackview', 'xgody']
all_brands = []
error_brand = []
i = 0
for index, row in data.iterrows():
    smartphone_brand = []
    for brand in brands:
        if str(row['brand']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['product_title']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['model']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
        elif str(row['specs']).lower().find(brand) != -1:
            smartphone_brand.append(brand)
    if len(smartphone_brand) == 0:
        smartphone_brand.append(None)
    elif smartphone_brand[0] == 'iphone':
        smartphone_brand[0] = 'apple'
    #Assign the brand that was most common
    all_brands.append(mode(smartphone_brand))
    smartphone_brand = set(smartphone_brand)
    if 'iphone' in smartphone_brand:
        smartphone_brand = smartphone_brand.remove('iphone')
    if (smartphone_brand != None) and (len(smartphone_brand) > 1):
        error_brand.append(row['asin_url'])
    if i%10000 == 0:
        print(f"{round(i/len(data),3)*100}%")
    i += 1
data['brand_overall'] = all_brands
set(error_brand)

0.0%
7.1%
14.2%
21.3%
28.499999999999996%
35.6%
42.699999999999996%
49.8%
56.89999999999999%
64.0%
71.1%
78.2%
85.39999999999999%
92.5%
99.6%


<ipython-input-36-6f4a83d35ef0>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['brand_overall'] = all_brands


{'https://www.amazon.com/dp/B01M24QYPN',
 'https://www.amazon.com/dp/B01MEF0R7P',
 'https://www.amazon.com/dp/B0747WWP4D',
 'https://www.amazon.com/dp/B077F17R8H',
 'https://www.amazon.com/dp/B07CCNTHYH',
 'https://www.amazon.com/dp/B07GBRB55Z',
 'https://www.amazon.com/dp/B07PHTYGY7',
 'https://www.amazon.com/dp/B085TCRFST',
 'https://www.amazon.com/dp/B087B53G76',
 'https://www.amazon.com/dp/B088MKVYTF',
 'https://www.amazon.com/dp/B08CH5JY5D',
 'https://www.amazon.com/dp/B08CH6X6M5',
 'https://www.amazon.com/dp/B08SV2Y7J6',
 'https://www.amazon.com/dp/B091JHNTNH',
 'https://www.amazon.com/dp/B094PNWZ5L',
 'https://www.amazon.com/dp/B0977LJRSM',
 'https://www.amazon.com/dp/B09CLMSZ1C',
 'https://www.amazon.com/dp/B09CLN4LLN',
 'https://www.amazon.com/dp/B09CLPL1R1',
 'https://www.amazon.com/dp/B09CLQBB6X',
 'https://www.amazon.com/dp/B09CLQDTFG',
 'https://www.amazon.com/dp/B09FVCCR7B',
 'https://www.amazon.com/dp/B09PWL7PYM'}

In [ ]:
['https://www.amazon.com/dp/B09QMR387G',
'https://www.amazon.com/dp/B06X1B444W',
 'https://www.amazon.com/dp/B09PN73JFT',
 'https://www.amazon.com/dp/B08HBPXM97',
 'https://www.amazon.com/dp/B0967KLCCG',
 'https://www.amazon.com/dp/B08FYLJK6B',
 'https://www.amazon.com/dp/B08D71WXYN',
 'https://www.amazon.com/dp/B07NPMHSHD',
 'https://www.amazon.com/dp/B08975Q8GK',
 'https://www.amazon.com/dp/B09QQPTRYB',
 'https://www.amazon.com/dp/B08QYSD3HW',
 'https://www.amazon.com/dp/B07RB2K1MF',
 'https://www.amazon.com/dp/B08QMT1G8P',
 'https://www.amazon.com/dp/B09CPHS9Q8',
 'https://www.amazon.com/dp/B09QCWT3HG',
 'https://www.amazon.com/dp/B09JCH6V5C',
 'https://www.amazon.com/dp/B08XKGSD1Y',
 
]

In [53]:
for i in data[data['brand_overall'].isnull()].asin_url.unique():
    print(i)

https://www.amazon.com/dp/B09V5S7ZNP
https://www.amazon.com/dp/B08WWQWGDN
https://www.amazon.com/dp/B08WVF25JC
https://www.amazon.com/dp/B0B3F9BRF9
https://www.amazon.com/dp/B08Z82YJ35
https://www.amazon.com/dp/B0759QPPKN
https://www.amazon.com/dp/B08XM9T5HV
https://www.amazon.com/dp/B09QMR387G
https://www.amazon.com/dp/B08XM6WG6H
https://www.amazon.com/dp/B09QV4HVD4
https://www.amazon.com/dp/B06X1B444W
https://www.amazon.com/dp/B09PN73JFT
https://www.amazon.com/dp/B08HBPXM97
https://www.amazon.com/dp/B0967KLCCG
https://www.amazon.com/dp/B07VCB7P1L
https://www.amazon.com/dp/B08Z7YLN53
https://www.amazon.com/dp/B09NY1NNDB
https://www.amazon.com/dp/B09G381X39
https://www.amazon.com/dp/B08RZ43CYV
https://www.amazon.com/dp/B0B5DP53D9
https://www.amazon.com/dp/B08FYLJK6B
https://www.amazon.com/dp/B08D71WXYN
https://www.amazon.com/dp/B07PT3R6P6
https://www.amazon.com/dp/B088VSPDFC
https://www.amazon.com/dp/B08XB51L99
https://www.amazon.com/dp/B07P8YZBG6
https://www.amazon.com/dp/B07ZHNVL55
h

# Iphone variable

In [37]:
iphone = []
for index, row in data.iterrows():
    if str(row['brand']).strip() == 'Apple':
        iphone.append(1)
    elif 'apple' in row['product_title'].lower():
        iphone.append(1)
    elif 'iphone' in row['product_title'].lower():
        iphone.append(1)
    else:
        iphone.append(0)
data['iphone'] = iphone

<ipython-input-37-cfdb310c83bf>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['iphone'] = iphone


In [56]:
data.iloc[-10000,]['date'] > 'May 14, 2022'

True

In [53]:
identifiers = []
for index, row in data.iterrows():
    if "formatType=current_format" in row['asin_url']:
        identifier = row['asin_url'].split('/')[3]
    else:
        identifier = row['asin_from_link']
    identifiers.append(identifier)
data['identifier'] = identifiers

<ipython-input-53-296bbbd8fc9c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['identifier'] = identifiers


In [167]:
data[(data.parent_asin == np.nan)]

,asin_url,brand,model,link,product_title,name,rating,review_title,country,date_amazon,specs,first_spec,review,helpful,picture,verified,asin_from_link,date,variant,renewed,brand_overall,iphone,identifier,uniq_color_parent_time,parent_asin,asin


# Uniqueness

In [168]:
#based on color instead of version asin
uniq_color_parent_times = []
size_dataset = len(data)
i = 0
for index, row in data.iterrows():
    date_review = row['date']
    asin_url = row['parent_asin']
    color = row['variant']
    variant = len(data[(data.variant == color) & (data.date <= date_review) & (data.parent_asin == asin_url)])
    parent = len(data[(data.parent_asin == asin_url) & (data.date <= date_review)])

    if parent == 0:
        uniq_color_parent_time = None
    else:
        uniq_color_parent_time = variant/parent
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{size_dataset}")
        print(uniq_color_parent_time)
        
    uniq_color_parent_times.append(uniq_color_parent_time)
data['uniq_color_parent_time'] = uniq_color_parent_times

1000/140595
1.0
2000/140595
1.0
3000/140595
1.0
4000/140595
0.7975077881619937
5000/140595
1.0
6000/140595
0.2652005174644243
7000/140595
1.0
8000/140595
0.5552884615384616
9000/140595
0.5639629831246598
10000/140595
0.3719533733663017
11000/140595
0.11143006263048016
12000/140595
0.5081763610018629
13000/140595
1.0
14000/140595
1.0
15000/140595
1.0
16000/140595
1.0
17000/140595
0.7606217616580311
18000/140595
1.0
19000/140595
0.6287015945330297
20000/140595
0.32289983844911146
21000/140595
0.15219421101774042
22000/140595
0.3288706542863312
23000/140595
0.6445530726256983
24000/140595
0.33941605839416056
25000/140595
1.0
26000/140595
0.4051150895140665
27000/140595
0.4553855959209688
28000/140595
0.3297609233305853
29000/140595
1.0
30000/140595
0.5712187958883994
31000/140595
1.0
32000/140595
0.6224899598393574
33000/140595
0.3286573146292585
34000/140595
0.3548795944233207
35000/140595
0.9056603773584906
36000/140595
0.3465085638998682
37000/140595
0.7209821428571429
38000/140595
0.2

In [5]:
len(data)

138074

In [6]:
#based on full dataset instead of only parent
uniq_color_data_times = []
size_dataset = len(data)
i = 0
for index, row in data.iterrows():
    date_review = row['date']
    color = row['variant']
    variant = len(data[(data.variant == color) & (data.date <= date_review)])
    data_at_date = len(data[data.date <= date_review])

    if data_at_date == 0:
        uniq_color_data_time = None
    else:
        uniq_color_data_time = variant/data_at_date
    i += 1
    if i % 1000 == 0:
        print(f"{i}/{size_dataset}")
        print(uniq_color_data_time)
        
    uniq_color_data_times.append(uniq_color_data_time)
data['uniq_color_data_time'] = uniq_color_data_times

1000/138074
0.017941484134783022
2000/138074
0.04920413968407674
3000/138074
0.10222634508348794
4000/138074
0.4301162982143659
5000/138074
0.47524199553239016
6000/138074
0.005648634745155792
7000/138074
0.10296697829104146
8000/138074
0.12077847738981111
9000/138074
0.1293529053602999
10000/138074
0.43152685105010274
11000/138074
0.063829237185895
12000/138074
0.4515942984262997
13000/138074
0.20744186046511628
14000/138074
0.101000147080453
15000/138074
0.03756040415873481
16000/138074
0.10248056006395194
17000/138074
0.4569204152249135
18000/138074
0.43152700639479563
19000/138074
0.4321229522040196
20000/138074
0.0184086414118363
21000/138074
0.052803277444806916
22000/138074
0.028903082142830275
23000/138074
0.46269938650306747
24000/138074
0.4303868986042366
25000/138074
0.43025473263151365
26000/138074
0.06511755971404128
27000/138074
0.030538624206990918
28000/138074
0.43231454430302163
29000/138074
0.45305188199389623
30000/138074
0.43193101901073927
31000/138074
0.0884454277

In [4]:
data['uniq_color_data_time'] = uniq_color_data_times

# Get asin version

In [47]:
asin_info = pd.read_csv("../../gen/audit/dict_asins_from_keepa_full.csv", sep = ";", header = None)
asin_info.columns = ['parent_asin', 'child_asin', 'spec1', 'spec2', 'spec3', 'spec4']
asin_info

,parent_asin,child_asin,spec1,spec2,spec3,spec4
0,B097M74GSR,B084GXSRL8,256GB,Red,Verizon,Renewed
1,B097M74GSR,B084H2VLKC,256GB,Red,T-Mobile,Renewed
2,B097M74GSR,B07ZPJW2XH,64GB,White,Unlocked,Renewed
3,B097M74GSR,B081Y4SML7,128GB,Yellow,AT&T,Renewed
4,B097M74GSR,B084HM3XGJ,128GB,Yellow,T-Mobile,Renewed
5,B097M74GSR,B08GJ9K8QW,64GB,White,GSM Carriers,Renewed
6,B097M74GSR,B084GX5ZQ8,128GB,Yellow,Verizon,Renewed
7,B097M74GSR,B081Y5C68R,256GB,Red,AT&T,Renewed
8,B097M74GSR,B07ZPK3KWH,128GB,Green,Unlocked,Renewed
9,B097M74GSR,B084H2JKK1,256GB,Purple,T-Mobile,Renewed


In [60]:
parent = asin_info[(asin_info['parent_asin']== asin) | (asin_info['child_asin']== asin)].iloc[0,0]
parent_child = {'parent_asin' : parent, 'asin' : asin}
parent_child

{'parent_asin': 'B09Q7VXHFH', 'asin': 'B07D3Z9ZST'}

In [66]:
data.columns

Index(['asin_url', 'brand', 'model', 'link', 'product_title', 'name', 'rating',
       'review_title', 'country', 'date', 'specs', 'first_spec', 'review',
       'helpful', 'picture', 'verified', 'date_trans', 'asin_from_link',
       'variant', 'renewed', 'brand_overall'],
      dtype='object')

## Get parent asin

In [61]:
asins = data['asin_from_link'].unique()
parents = []
missing_parent = []
for asin in asins:
    try:
        parent = asin_info[(asin_info['parent_asin']== asin) | (asin_info['child_asin']== asin)].iloc[0,0]
    except:
        parent = None
        missing_parent.append(asin)
    parent_child = {'parent_asin' : parent, 'asin' : asin}
    parents.append(parent_child)
parent_child = pd.DataFrame(parents)

data = data.merge(parent_child, left_on='asin_from_link', right_on = 'asin', how='left')

In [79]:
sum(data['parent_asin'].isnull())

1695

# Get version asin

In [80]:
def retrieve_version_asin(potential_asins, test_string):
    potential_version_asins = []
    for index,row in potential_asins.iterrows():
        specs = [x.lower().replace(' ', '') for x in row.values.tolist()[2:] if x != "0"]
        if len([x for x in specs if x in test_string.replace(' ', '').lower()]) == len(specs):
            potential_version_asins.append(row['child_asin'])
    return potential_version_asins

In [81]:
i = 1
length = len(data)
version_asins = []
for s_index, s_row in data.iterrows():
    #If the comments are filtered on current format, it means the version asin is in the link
    if "formatType=current_format" in row['asin_url']:
        version_asin = row['asin_url'].split('/')[-2]
    #Else it has to be searched for in the specs or title
    else:
        #Get a list of all possible asins by filtering on the parent asin
        potential_asins = asin_info[asin_info['parent_asin'] == s_row['parent_asin']]
        #If there's only one potential asin, choose that one
        if len(potential_asins) == 1:
            potential_version_asins = [potential_asins['child_asin'].tolist()[0]]
        #Else look in the specs
        else:
            test_string = s_row['specs']
            potential_version_asins = retrieve_version_asin(potential_asins, test_string)
        #If there's nothing. Look in the product title
        if len(potential_version_asins) == 0:
            print("looking in title...")
            test_string = s_row['product_title']
            potential_version_asins = retrieve_version_asin(potential_asins, test_string)
        elif len(potential_version_asins) > 1:
            print(f"multiple asins found: {potential_version_asins}")
        #Take first item of the list of potential asins
        if  len(potential_version_asins) >= 1:
            version_asin = potential_version_asins[0]
        #If nothing found. Impute none
        else:
            version_asin = None
    if i % 10000 == 0:
        print(f"{i}/{length}")
    if version_asin == "B":
        print('something went wrong')
    i += 1
    version_asins.append(version_asin)

looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
multiple asins found: ['B07P978C2R', 'B08BGD4G36']
multiple asins found: ['B07P978C2R', 'B08BGD4G36']
multiple asins found: ['B07P978C2R', 'B08BGD4G36']
multiple asins found: ['B07P978C2R', 'B08BGD4G36']
looking in title...
multiple asins found: ['B07P83CBGR', 'B08BGCHXR1']
multiple asins found: ['B07P978C2R', 'B08BGD4G36']
multiple asins found: ['B08BGFCPS4', 'B07P9765VS']
multiple asins found: ['B08BG9LS24', 'B07P6Y7954']
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in t

looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


looking in title...
looking in title...
looking in title...
multiple asins found: ['B08PP43P1V', 'B09JFGWDK9']
looking in title...
looking in title...
multiple asins found: ['B08PP43P1V', 'B09JFGWDK9']
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title..

...

looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW',

looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
multiple asins found: ['B07759CGRW', 'B07RV5J8CZ']
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in

looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...
looking in title...


In [89]:
data['asin'] = version_asins

## version asin corrections based on parent asin

In [11]:
version_asin_corrections = [
    ["B0BGZ4P8JT", 'Style: Motorola One 5G AceColor: SilverSize: 128GB', "B08NWD7K8H"],
    ["B0BL9TKSBL", 'Color: Moonlight GraySize: 256GBPattern Name: Moto G 5G', "B09TZ5ZBJT"],
    ["B0BKYJJKS3", 'Color: Midnight BlackConfiguration: 16GB RAMDigital Storage Capacity: 256 GBConnectivity Technology: 5G', "B0BBP4HCSN"],
    ["B0BLT3L8WF", 'Color: Volcanic BlackSize: 8GB RAM + 128GB storage', "B09S8BV4D9"],
    ["B0BLT3L8WF", 'Color: Emerald ForestSize: 8GB RAM + 128GB storage', "B09S8MWLSG"],
    ["B0BLT3L8WF", 'Color: Volcanic BlackSize: 12GB RAM+256GB storage', "B07XM54RWB"],
    ["B086R92XM1", 'Wireless Provider: UnlockedColor: Aura GlowMemory Storage Capacity: 256.0 GB', "B086R92XM1"],
    ["B0B5FHVMCJ", 'Color: Burgundy', "B0B6WNSGDK"],
    ["B0BF2YBYD1", 'Color: SC490', "B08D71WXYN"],
    ["B0B529LWQ3", 'Color: Burgundy', "B09WL8FBR5"],
    ["B0BG8SR1N7", 'Color: Morning Mist', "B093GS36QT"],
    ["B0BG8SR1N7", 'Color: Green', "B09BZVKT6Q"],
    ["B0BG8G11GH", 'Color: green', "B09M2J31NZ"],
    ["B0BG8G11GH", 'Color: gold', "B09M2VXMQV"],
    ["B0BG8G11GH", 'Color: black', "B09M2XRQN7"],
    ["B0BJSCM58P", 'Color: Black', "B09P6154ZT"],
    ["B0BJSCM58P", 'Color: Orange', "B09P61PH7N"],
    ["B0BLR46Y13", "['Size: 512 GB', 'Color: Cosmos Blue']", "B09TG89P52"],
    ["B0BLR46Y13", "['Size: 512 GB', 'Color: Stardust White']", "B09TG61BFL"],
    ["B09Q7VXHFH", "['Size: 64GB', 'Color: Coral Blue', 'Service Provider: AT&amp;T', 'Product grade: Renewed']", "B07BFBWS8D"],
    ["B09Q7VXHFH", "['Size: 64GB', 'Color: Lilac Purple', 'Service Provider: AT&amp;T', 'Product grade: Renewed']", "B09Q7VXHFH"],
    ["B09Q7VXHFH", "['Size: 64GB', 'Color: Midnight Black', 'Service Provider: AT&amp;T', 'Product grade: Renewed']", "B07BFDK78Q"],
    ["B0BLT3L8WF", "Color: Volcanic BlackSize: OP10 Pro 12GB + 256GB|Buds Pro Earbuds", "B07XM54RWB"],
    ["B0BG8G11GH", "Color: S22+ Black", "B09M2XRQN7"],
    ["B0B49PN346", "Color: Stellar Black 8+128G", 'B0B2J6BG7S'],
    ["B0BGG1ZKSP", "Color: 7 Red&Blue 16+256 GB", "B09S5JQNXH"],
    ["B0BMLM7F1C", "Style: Power Armor 14 Pro", "B0B73VD726"],
    ["B0BMLM7F1C", "Style: Power Armor 14", "B09M3NN53L"],
    ["B09MHJZ955", "Color: BS4 Pro-12+256GB-Black", "B09RZMR2ST"]
]

for version_asin in version_asin_corrections:
    start = len(data.loc[(data.parent_asin == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'])
    data.loc[(data.parent_asin == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'] = version_asin[2]
    end = len(data.loc[(data.parent_asin == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'])
    print(f"replaced {start-end} values")



replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 0 values
replaced 30 values
replaced 10 values
replaced 20 values
replaced 1 values
replaced 1 values
replaced 1 values
replaced 11 values
replaced 6 values
replaced 6 values
replaced 3 values


## version asin corrections based on link

In [13]:
version_link_corrections = [
    ["https://www.amazon.com/dp/B08YNC8X92", "Size: BV4900Color: Green", "B08YNC8X92"],
    ["https://www.amazon.com/dp/B08YNC8X92", "Size: BV4900Color: Black", "B08YN5QNQX"],
    ["https://www.amazon.com/dp/B0B5ZZPC9C", "Size: 12GB+256GBColor: Black", "B0B613D7YM"],
    ["https://www.amazon.com/dp/B08YN5PRNG", "Wireless Provider: T-MobileColor: A80-BlueMemory Storage Capacity: 16.0 GB", "B08ZMRCZ5J"],
    ["https://www.amazon.com/dp/B08YN5PRNG", "Wireless Provider: T-MobileColor: A80-RedMemory Storage Capacity: 16.0 GB", "B092LGRRG1"],
    ["https://www.amazon.com/dp/B09GVQP857", "Color: Celestial Blue", "B09GVSTT84"],
    ["https://www.amazon.com/dp/B09GVQP857", "Color: Moonlight White", "B09GVQP857"],
    ["https://www.amazon.com/dp/B09GVQP857", "Color: Meteorite Gray", "B09MBCBF77"],
    ["https://www.amazon.com/dp/B09BZD5NJN", "Color: Blue", "B09BZD5NJN"],
    ["https://www.amazon.com/dp/B09BZD5NJN", "Color: Black", "B09PH729N1"],
    ["https://www.amazon.com/dp/B09WKFCKL2", "Color: Gray", "B09WMWF7PK"],
    ["https://www.amazon.com/dp/B09W24VW1T", "Size: 4GB+64GBColor: Sunglow Gold", "B09W231C2C"],
    ["https://www.amazon.com/dp/B09W24VW1T", "Size: 4GB+64GBColor: Galaxy Blue", "B09W23PQJY"],
    ['https://www.amazon.com/dp/B09W24VW1T', "Size: 4GB+64GBColor: Starry Black", "B09W239NNF"]
]


for version_asin in version_link_corrections:
    start = len(data.loc[(data.asin_url == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'])
    data.loc[(data.asin_url == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'] = version_asin[2]
    end = len(data.loc[(data.asin_url == version_asin[0]) &
             (data.asin.isnull()) &
             (data.specs == version_asin[1]), 'asin'])
    print(f"replaced {start-end} values")


replaced 6 values
replaced 10 values
replaced 1 values
replaced 56 values
replaced 63 values
replaced 13 values
replaced 5 values
replaced 14 values
replaced 2 values
replaced 4 values
replaced 1 values
replaced 2 values
replaced 10 values
replaced 2 values


In [94]:
data.to_csv("../../gen/audit/data_before_keepa.csv", sep = ";")

## Merging with keepa data

In [7]:
keepa_data = pd.read_csv("../../gen/audit/keepa_updated_full_dataset.csv", sep = ";", index_col = 0)
keepa_data

,price_new,price_used,rating,reviews,asin,oos_new,oos_used,listed_since
2021-04-06,629.990000,NaN,NaN,NaN,B08WF93R89,7,80,5317920
2021-04-18,650.240000,NaN,NaN,NaN,B08WF93R89,7,80,5317920
2021-05-04,629.990000,NaN,NaN,NaN,B08WF93R89,7,80,5317920
2021-05-06,679.990000,NaN,NaN,NaN,B08WF93R89,7,80,5317920
2021-05-08,670.073333,NaN,NaN,NaN,B08WF93R89,7,80,5317920
...,...,...,...,...,...,...,...,...
2022-11-01 00:00:00,NaN,NaN,NaN,73.0,B09W231C2C,8,100,5981760
2022-11-03 00:00:00,NaN,NaN,3.8,NaN,B09W231C2C,8,100,5981760
2022-11-25 00:00:00,NaN,NaN,3.7,85.0,B09W231C2C,8,100,5981760
2022-11-28 00:00:00,NaN,NaN,NaN,86.0,B09W231C2C,8,100,5981760


In [33]:
keepa_data.index = pd.to_datetime(keepa_data.index)

In [2]:
data = pd.read_csv("../../gen/audit/data_before_keepa.csv", sep = ";", index_col = 0)

In [76]:
data.columns

Index(['asin_url', 'brand', 'model', 'link', 'product_title', 'name', 'rating',
       'review_title', 'country', 'date', 'specs', 'first_spec', 'review',
       'helpful', 'picture', 'verified', 'asin_from_link', 'date_trans',
       'variant', 'renewed', 'brand_overall', 'iphone', 'identifier',
       'uniq_color_parent_time', 'parent_asin'],
      dtype='object')

In [19]:
data['date'] = pd.to_datetime(data['date'])

In [20]:
data = data.sort_values(by = ['asin', 'date'], ascending = [True, True])

In [26]:
def merge_data_with_keepa(asin, data):
    keepa_subset = keepa_data[keepa_data.asin == asin]
    if len(keepa_subset) == 0:
        df = keepa_subset
    else:
        keepa_subset = pd.merge(pd.DataFrame(pd.date_range(min(keepa_subset.index), max(keepa_subset.index), freq= 'D')).rename(columns = {0:"date"}),
                                 keepa_subset, how="left",  left_on='date', right_index=True).set_index("date")
        keepa_subset = keepa_subset.replace(-1, np.nan)
        keepa_subset = keepa_subset.sort_index()
        keepa_subset = keepa_subset.interpolate(method='time')
        keepa_subset = keepa_subset.bfill()
        keepa_subset = keepa_subset.ffill()
        keepa_subset = keepa_subset.rename_axis('date').reset_index()
        df = pd.merge_asof(left = data[data.asin == asin], right = keepa_subset, on = 'date', direction = "nearest")
    return df

In [69]:
pd.DataFrame(pd.date_range(min(keepa_subset.index), max(keepa_subset.index), freq= 'D')).rename(columns = {0:"date"})

,date
0,2019-11-27
1,2019-11-28
2,2019-11-29
3,2019-11-30
4,2019-12-01
...,...
1100,2022-12-01
1101,2022-12-02
1102,2022-12-03
1103,2022-12-04


In [88]:
merge_data_with_keepa("B07ZLGR1B1")


,asin_url,brand,model,link,product_title,name,rating_x,review_title,country,date_amazon,...,parent_asin,asin_x,price_new,price_used,rating_y,reviews,asin_y,oos_new,oos_used,listed_since
0,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Amazon Customer,5.0,So very amazing,the United States,"May 20, 2020",...,B097M74GSR,B07ZLGR1B1,729.980857,439.000000,4.000000,1.000000,B07ZLGR1B1,18.0,89.0,4639680.0
1,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Dilma Perrud,1.0,Everything thinks,the United States,"June 2, 2020",...,B097M74GSR,B07ZLGR1B1,729.973429,439.000000,3.554545,3.454545,B07ZLGR1B1,18.0,89.0,4639680.0
2,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",GK,5.0,Excelente condición,the United States,"June 9, 2020",...,B097M74GSR,B07ZLGR1B1,729.990000,439.000000,3.320000,4.400000,B07ZLGR1B1,18.0,89.0,4639680.0
3,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Alex,5.0,Battery life,the United States,"June 21, 2020",...,B097M74GSR,B07ZLGR1B1,826.990000,439.000000,3.671429,5.857143,B07ZLGR1B1,18.0,89.0,4639680.0
4,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Jeremiah Wilkerson,4.0,I really don't know whether item is like but ...,the United States,"July 5, 2020",...,B097M74GSR,B07ZLGR1B1,826.656667,439.000000,3.864706,8.294118,B07ZLGR1B1,18.0,89.0,4639680.0
5,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Michael Johnson,1.0,Don’t buy,the United States,"July 15, 2020",...,B097M74GSR,B07ZLGR1B1,824.990000,439.000000,3.923529,10.411765,B07ZLGR1B1,18.0,89.0,4639680.0
6,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Shamarie Williams,1.0,The phone is not waterproof. It appears the ph...,the United States,"July 30, 2020",...,B097M74GSR,B07ZLGR1B1,789.990000,439.000000,3.950000,16.000000,B07ZLGR1B1,18.0,89.0,4639680.0
7,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",GDianned,5.0,iPhone 11 - like new!,the United States,"September 7, 2020",...,B097M74GSR,B07ZLGR1B1,737.990000,439.000000,4.188889,27.750000,B07ZLGR1B1,18.0,89.0,4639680.0
8,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Ronald C.,3.0,There’s a problem with this device,the United States,"September 30, 2020",...,B097M74GSR,B07ZLGR1B1,674.990000,439.000000,4.090476,39.666667,B07ZLGR1B1,18.0,89.0,4639680.0
9,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,apple,IPhone 11,https://www.amazon.com/Apple-iPhone-11-64GB-Bl...,"Apple iPhone 11, 256GB, Red for Verizon (Renewed)",Trex,5.0,Just as expected,the United States,"October 25, 2020",...,B097M74GSR,B07ZLGR1B1,689.988750,439.000000,4.000000,50.666667,B07ZLGR1B1,18.0,89.0,4639680.0


In [170]:
#Looping over rows data to merge each keepa and data for each asin seperately
unique_asins = [x for x in list(set(data.asin.unique()) - set(data_overall.asin_x.unique())) if str(x) != 'nan']
data_overall
i = 1
length = len(unique_asins)
for asin in unique_asins:
    print(f"{asin} : {i}/{length}")
    print(len(data_overall))
    df = merge_data_with_keepa(str(asin))
    if len(df) == 0:
        print(f'no keepa data found for {asin}')
    else:
        data_overall = pd.concat([data_overall, df])
    i += 1

B08KYX5RSR : 1/1903
37
B08DFWD3ML : 2/1903
82
B07PXSNRYY : 3/1903
130
B01L5PBYQC : 4/1903
478
B09LKWZFNV : 5/1903
513
B09WYLJBJZ : 6/1903
514
B09KNLMVM7 : 7/1903
517
B08KYH35ZR : 8/1903
523
B07P9765VS : 9/1903
559
B08PNV737N : 10/1903
1361
B08FCDP4QC : 11/1903
1362
B07GZLLTWC : 12/1903
1384
B09YL7PDX6 : 13/1903
1401
B07MZFZSGN : 14/1903
1402
B09PCD2VCC : 15/1903
1439
B07GT5TVPQ : 16/1903
1440
B0B6T367MH : 17/1903
1804
B09CKZ9TRQ : 18/1903
1812
B09P1W6NLN : 19/1903
1824
B07ZZKHSW6 : 20/1903
1826
B07T3Y7BVD : 21/1903
1840
B08BJ2SQV3 : 22/1903
1850
B085QMXBD2 : 23/1903
1861
B08VRGC171 : 24/1903
1866
B09WCNQVZS : 25/1903
1876
B08BX7LJ5T : 26/1903
1929
B07K8RNW7H : 27/1903
2001
B0B7SL4GFS : 28/1903
2739
B07TZRM255 : 29/1903
2741
B09KCNZKX3 : 30/1903
2745
B07RFF3LMF : 31/1903
2769
B08PNKTGD3 : 32/1903
2776
B09PN73JFT : 33/1903
2780
B07N9MF1Q3 : 34/1903
2825
B09LP862LG : 35/1903
3073
B07T8D3LM4 : 36/1903
3076
B096YJTGK9 : 37/1903
3080
B00NWX6TCU : 38/1903
3082
B081QT6R64 : 39/1903
3086
B077H7

B0B2DTYN9C : 305/1903
18479
B07ZQRW6N1 : 306/1903
18480
B08CJFTN83 : 307/1903
18492
B09LNWB9LL : 308/1903
18500
B07YYM9J4L : 309/1903
18501
B08Q1YWQQ9 : 310/1903
18640
B09SM24S8C : 311/1903
18746
B097C3TS3Q : 312/1903
18750
B07ZZK5BDF : 313/1903
18781
B07V3TL48P : 314/1903
18782
B08LXKCCGL : 315/1903
19023
B07TGTBDMK : 316/1903
19027
B07BSTPWTS : 317/1903
19090
B07YMJ9WFN : 318/1903
19152
B08L9QQ6RZ : 319/1903
19174
B08BHN96L6 : 320/1903
19194
B07QZJ5TC4 : 321/1903
19215
B087HP47FD : 322/1903
19245
B088MMXN3X : 323/1903
19271
B08PP43P1V : 324/1903
19286
B08KL7BRJB : 325/1903
19291
B07SH9JWBF : 326/1903
19294
B084GX6HR2 : 327/1903
19328
B081Y5WL5F : 328/1903
19347
B076M9RQJ2 : 329/1903
19353
B09LNX6KQS : 330/1903
19802
B09P484865 : 331/1903
19823
B07ZQYZ82X : 332/1903
19826
B096YGJ2TF : 333/1903
19852
B089LKV86J : 334/1903
19856
B08723759H : 335/1903
19955
B09KNY65MZ : 336/1903
20141
B07T2MGC85 : 337/1903
20142
B07D9V66S3 : 338/1903
20143
B07WK8HVW1 : 339/1903
20193
B097QDTNM5 : 340/190

B07PL5PD7V : 598/1903
44375
B07XFGHVFY : 599/1903
44409
B081TLXWVT : 600/1903
44789
B094FBCB2K : 601/1903
44851
B096YGNXZ5 : 602/1903
44852
B07W6L5G3R : 603/1903
44861
B08HVVCBHL : 604/1903
44879
B09SHYDC22 : 605/1903
45197
B08X6VDLQL : 606/1903
45198
B09B79YT3S : 607/1903
45213
B09GGD1T6K : 608/1903
45316
B08H4RFTNQ : 609/1903
45322
B08KTHNWX3 : 610/1903
45356
B09LNW3CY2 : 611/1903
45386
B08N5VPCFL : 612/1903
45407
B09YS5X3B4 : 613/1903
45422
B09JFKQ6Y4 : 614/1903
45425
B07YMM3P8V : 615/1903
45427
B07ZQRN3KR : 616/1903
45517
B09X36FCD3 : 617/1903
45532
B0775MV9K4 : 618/1903
45565
B09LP8D3WV : 619/1903
45647
B081Y4DMSG : 620/1903
45648
B09SK27K2P : 621/1903
45650
B09BDJFBMV : 622/1903
45653
B086T69CLF : 623/1903
45659
B096YJ8YTP : 624/1903
45662
B0939K5M9Q : 625/1903
45672
B07K1GJGVC : 626/1903
45759
B07T9H8QL7 : 627/1903
45826
B07NPMHSHD : 628/1903
45829
B07KBB83N8 : 629/1903
45925
B07SQB29JS : 630/1903
45936
B074VFRKZG : 631/1903
45939
B08GJB6KD3 : 632/1903
46052
B08Y64KRTF : 633/190

B08BHYXFNW : 891/1903
63550
B09295KQ9Q : 892/1903
63551
B08PNKR7M9 : 893/1903
63733
B076XLLCQC : 894/1903
63735
B0841NBKNP : 895/1903
64552
B09XXGMSCM : 896/1903
64587
B09VWWQCDP : 897/1903
64590
B07RDMLR4V : 898/1903
64591
B081TJ7B88 : 899/1903
64620
B0891VNDNT : 900/1903
64883
B07ZZMRV8F : 901/1903
64885
B09TWSY6V4 : 902/1903
64888
B08D71WXYN : 903/1903
64889
B09MMNWWDQ : 904/1903
65069
B09181P24R : 905/1903
65074
B0979KD57W : 906/1903
65080
B08GL5CWNS : 907/1903
65102
B08J4JYD47 : 908/1903
65189
B08Q1RN311 : 909/1903
65366
B07WRF8PS6 : 910/1903
65408
B08M3V5KGS : 911/1903
65418
B07MQVW2T6 : 912/1903
65436
B07ZQSJRGL : 913/1903
65474
B088N4WXGY : 914/1903
65494
B08FYTSXGQ : 915/1903
65517
B07R53GJK2 : 916/1903
66299
B07FK3HGWL : 917/1903
66339
B07Z44RDGC : 918/1903
66382
B09495Q1H7 : 919/1903
66390
B07RJKY76Z : 920/1903
66675
B07772QJ2G : 921/1903
66676
B07YMFYTYY : 922/1903
66745
B07C65XFBB : 923/1903
66835
B078P38R8L : 924/1903
66979
B09M8RHYDJ : 925/1903
67003
B08Y67CMB9 : 926/190

B08Z7R6KK7 : 1177/1903
85872
B09V5S7ZNP : 1178/1903
85879
B094XJT8LS : 1179/1903
85901
B0787SDRQ8 : 1180/1903
85902
B08PNRYBVB : 1181/1903
86046
B08SJ2TWTX : 1182/1903
86047
B0B5VSP7X9 : 1183/1903
86055
B081Y674B3 : 1184/1903
86056
B08X3SKF4S : 1185/1903
86057
B07ZPKN6YR : 1186/1903
86118
B07KMCQWT4 : 1187/1903
87139
B08PN6ZMY8 : 1188/1903
87147
B08PNWLWTC : 1189/1903
87149
B09CLPL1R1 : 1190/1903
87151
B07R71ZQYP : 1191/1903
87168
B09MZCWH4J : 1192/1903
87189
B09237HBWJ : 1193/1903
87209
B08723GJ2T : 1194/1903
87211
B09WZW47LP : 1195/1903
87409
B097VNQFLW : 1196/1903
87418
B08KZ8DYQW : 1197/1903
87420
B08DG27BTG : 1198/1903
87447
B09LKXHWCF : 1199/1903
87515
B0B879S4RP : 1200/1903
87550
B07BFCZYRM : 1201/1903
87551
B08HKRVZ4W : 1202/1903
87717
B01CYYYRNK : 1203/1903
87733
B07P839TX1 : 1204/1903
88058
B087XSTN2X : 1205/1903
88344
B09XFDCK7S : 1206/1903
88345
B08BGD4G36 : 1207/1903
88347
B09B1NY84G : 1208/1903
88374
B085HY7M51 : 1209/1903
88460
B08LQTQRJR : 1210/1903
88465
B08V1X89P9 : 1

B08CGXVPXW : 1456/1903
107533
B0792DWMS5 : 1457/1903
107571
B09NVTTJQJ : 1458/1903
107591
B08LP53S58 : 1459/1903
107595
B0B5VTB9ZY : 1460/1903
107652
B087XBPYWB : 1461/1903
107654
B09WDGRRJB : 1462/1903
107656
B07ZZLZPNK : 1463/1903
107657
B07VCB7P1L : 1464/1903
107658
B096T6279W : 1465/1903
107727
B08PP4X584 : 1466/1903
107732
B09PPP7CKV : 1467/1903
107763
B093RQCX2D : 1468/1903
107765
B084HMBDV6 : 1469/1903
107842
B085QNCWKW : 1470/1903
107843
B087YD8Q4N : 1471/1903
107851
B08GQ6JD9Y : 1472/1903
107853
B09LP7YLF9 : 1473/1903
107858
B07FMGTNQD : 1474/1903
107860
B088YN4RFQ : 1475/1903
107914
B08T1VR8QW : 1476/1903
107920
B09CJ6X43H : 1477/1903
108068
B07J1Q8P6G : 1478/1903
108072
B09PG8FQP3 : 1479/1903
108111
B08BCDWLGC : 1480/1903
108112
B096QQ5G1K : 1481/1903
108222
B09H22QBS5 : 1482/1903
108233
B0977LJRSM : 1483/1903
108254
B08D726WNL : 1484/1903
108256
B07L2GZDKG : 1485/1903
108302
B08PNBW45M : 1486/1903
108310
B09FDJG3XF : 1487/1903
108312
B09PFC2DVD : 1488/1903
108326
B0897Y6KNY

B0916FZ5CQ : 1730/1903
122444
B09GRYJ81V : 1731/1903
122450
B08YS71CLN : 1732/1903
122488
B09H2DG3LV : 1733/1903
122497
B07ZPJWGKZ : 1734/1903
122527
B09ZLLFNHW : 1735/1903
122664
B09RRNHVJQ : 1736/1903
122665
B07GVRC54W : 1737/1903
122667
B08PPLQB2P : 1738/1903
122721
B09TG2L2GY : 1739/1903
122732
B09TKZMGYT : 1740/1903
122733
B07VQWNQ9F : 1741/1903
122735
B09GW9KMHD : 1742/1903
122759
B07ZPKR714 : 1743/1903
122766
B08HBL6N91 : 1744/1903
122976
B09VCTP7ZL : 1745/1903
123037
B084GX4BVG : 1746/1903
123071
B08BHJ5XH9 : 1747/1903
123080
B09KMG6YL7 : 1748/1903
123092
B07VNP3ZCS : 1749/1903
123147
B088N6KHHD : 1750/1903
123161
B09LNT7B9D : 1751/1903
123326
B096T6JJ6J : 1752/1903
123327
B07D6TCG98 : 1753/1903
123329
B09BG4SW2N : 1754/1903
123454
B08PNHB9L7 : 1755/1903
123461
B088NMH5KL : 1756/1903
123490
B07ZQSQSBM : 1757/1903
123511
B09C6LMM28 : 1758/1903
124096
B099RPX1SG : 1759/1903
124132
B091JHNTNH : 1760/1903
124133
B08R5ZTT59 : 1761/1903
124157
B07JZCFFQR : 1762/1903
124161
B08PNN2VNR

In [173]:
len(data) - len(data_overall)

5701

In [73]:
data_overall = data_overall.rename(columns = {'rating_x' : 'rating_amazon', 'rating_y' : 'rating_keepa', 'asin_x' : 'asin'})

In [74]:
data_overall = data_overall.drop(columns = ['asin_y'])

In [75]:
print(len(data_overall))
data_overall_no_dupli = data_overall[~data_overall.duplicated(subset=['name', 'rating_amazon', 'date', 'review', 'identifier', 'specs'])]
print(len(data_overall_no_dupli))

3256
3256


In [4]:
len(data)

138074

In [3]:
data.isnull().sum(axis=0)

Unnamed: 0                     0
asin_url                       0
brand                          1
model                       1090
link                           0
product_title                  0
name                           9
rating_amazon                  0
review_title                  57
country                        0
date_amazon                 3256
specs                          0
first_spec                   128
review                       331
helpful                    19763
picture                     5333
verified                    5333
asin_from_link                 0
date                           0
variant                        0
renewed                        0
brand_overall               4444
iphone                         0
identifier                     0
uniq_color_parent_time       177
parent_asin                  177
asin                        3256
price_new                  13248
price_used                 15073
rating_keepa               13248
reviews   

# Get count of reviews by parent

In [180]:
data_overall_no_dupli.groupby(['parent_asin'])['parent_asin', 'reviews'].count().sort_values('reviews', ascending = False).head(50)

<ipython-input-180-d44008d79c20>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_overall_no_dupli.groupby(['parent_asin'])['parent_asin', 'reviews'].count().sort_values('reviews', ascending = False).head(50)


,parent_asin,reviews
parent_asin,,
B08L97BW9J,8886,8886
B086RM3DQS,7373,7373
B09HPPL95L,3388,3388
B089SRK8VQ,3124,3124
B08ZWJXK1Q,3006,3006
B097M74GSR,2705,2705
B08D969C67,2667,2667
B09Q7VXHFH,2609,2599
B07RWPFQLM,2193,2193


# Saving the data

In [ ]:
data[data['version_asin'].isna()].to_csv("../../gen/audit/Amazon_missing.csv", sep = ";")

In [6]:
data_overall['diff_rating'] = data_overall.rating_x - data_overall.rating

In [178]:
data_overall.describe().to_csv("../../gen/audit/descriptivestats_new.csv", sep = ";", decimal=",")

In [7]:
data_overall.to_csv("../../gen/temp/data_overall.csv", sep = ";")

# Brand market shares

In [34]:
brands = pd.read_csv("../../gen/temp/brand_shares.csv", index_col = 0)
brands

,brand,date,market_share,date_trans
1,acer,2012.01,0.00,2012-01-15
2,acer,2012.02,0.00,2012-02-15
3,acer,2012.03,0.00,2012-03-15
4,acer,2012.04,0.00,2012-04-15
5,acer,2012.05,0.00,2012-05-15
...,...,...,...,...
4404,zte,2022.06,0.06,2022-06-15
4405,zte,2022.07,0.06,2022-07-15
4406,zte,2022.08,0.06,2022-08-15
4407,zte,2022.09,0.05,2022-09-15


In [35]:
brands.groupby(["brand", "date"]).count().loc[brands.groupby(["brand", "date"]).count().market_share > 1, :]

,,date,market_share
brand,date_trans,,


In [36]:
brand_corrections = {"rim" : "blackberry",
                    "samesung" : "samsung",
                    "sumsung" : "samsung",
                    "infinex" : "infinix"}

for wrong, good in brand_corrections.items():
    brands.loc[(brands.brand == wrong), "brand"] = good


In [38]:
brands_corrected = brands.groupby(["brand", "date"]).sum()

In [41]:
brands_corrected.to_csv("../../gen/output/brand_corrected.csv", sep=";")